Mainstream Bias in Dynamic Recommendation

In [2]:
# Import dependencies
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppress TensorFlow logs
import warnings; warnings.simplefilter('ignore')  # Ignore warnings for cleaner output
# import time
import numpy as np
import argparse
# import utility
from Simulation_basic import Simulation
import pickle
import pandas as pd
# from tqdm import tqdm
from math import log
from scipy.sparse import coo_matrix
# import seaborn as sns
import matplotlib.pyplot as plt
# from scipy.stats import skew
# from scipy.stats import mode
# from sklearn.neighbors import LocalOutlierFactor

In [3]:
# Parse command-line arguments
# args = parser.parse_args()

# Define the default values for arguments
default_args = {
    'run': 1,
    'iteration': 1000,
    'exp': 1,
    'cycle_itr': 50,
    'epoch': 20,
    'K': 20,
    'lr': 0.001,
    'reg': 1e-5,
    'hidden': 100,
    'neg': 5,
    'data': 'ml1m'
}

# Create the args object with default values
args = argparse.Namespace(**default_args)

In [4]:
# Load truth data and set experiment parameters
truth = np.load('./Data/' + args.data + '/truth.npy')
args.num_user = truth.shape[0]
args.num_item = truth.shape[1]
audience_size = np.sum(truth, axis=0)
item_sorted = np.argsort(audience_size)
truth_like = list(np.load('./Data/' + args.data + '/user_truth_like.npy', allow_pickle=True))

# Print total truth for reference
print('')
print('!' * 30 + ' Total truth ' + str(np.sum(truth)) + ' ' + '!' * 30)
print('')


!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! Total truth 223869.0 !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!



In [5]:
# Calculate mainstream scores (MS_similarity) using your code
with open('./Data/' + args.data + '/info.pkl', 'rb') as f:
    info = pickle.load(f)
    num_user = info['num_user']
    num_item = info['num_item']

train_df = pd.read_csv('./Data/' + args.data + '/train_df.csv')

# Calculate user popularity
pos_user_array = train_df['userId'].values
pos_item_array = train_df['itemId'].values
train_mat = coo_matrix((np.ones(len(pos_user_array)), (pos_user_array, pos_item_array)), shape=(num_user, num_item)).toarray()
user_pop = np.sum(train_mat, axis=1)

# Calculate standard deviation of user interactions
user_stddev = np.std(train_mat, axis=1)

# Save the user standard deviations to a file (adjust the path accordingly)
with open(f'./Data/{args.data}/user_stddev.npy', "wb") as f:
    np.save(f, user_stddev)

# Calculate Jaccard similarity matrix
Jaccard_mat = np.matmul(train_mat, train_mat.T)
deno = user_pop.reshape((-1, 1)) + user_pop.reshape((1, -1)) - Jaccard_mat + 1e-7
Jaccard_mat /= deno
Jaccard_mat = Jaccard_mat + np.eye(num_user) * -9999
Jaccard_mat = Jaccard_mat[np.where(Jaccard_mat > -1)].reshape((num_user, num_user - 1))

# Calculate Mainstream Similarity (MS_similarity) by taking the mean along axis 1
MS_similarity = np.mean(Jaccard_mat, axis=1)

# Save the MS similarity to a file (adjust the path accordingly)
with open(f'./Data/{args.data}/MS_similarity.npy', "wb") as f:
    np.save(f, MS_similarity)

In [7]:
# Initialize a list to store Gini coefficients after each epoch
gini_coefficients = []

# Run the experiment for a specified number of runs
for r in range(args.run):
    print('')
    print('#' * 100)
    print('#' * 100)
    print(' ' * 50 + ' Experiment run ' + str(r + 1) + ' ' * 50)
    print('#' * 100)
    print('#' * 100)

    # Initialize the simulation with provided arguments and data
    simulation = Simulation(args, truth, truth_like)
    init_popularity = simulation.initial_iterations()  # Perform initial iterations to gather feedback
    gini_coefficients.append(simulation.run_simulation())  # Run the main simulation


####################################################################################################
####################################################################################################
                                                   Experiment run 1                                                  
####################################################################################################
####################################################################################################
Namespace(K=20, cycle_itr=50, data='ml1m', epoch=20, exp=1, hidden=100, iteration=1000, lr=0.001, neg=5, num_item=3406, num_user=1000, reg=1e-05, run=1)
****************************** Start initial random iterations ******************************
---------- Iteration 1 ----------


  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:00<00:00, 5952.46it/s]

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Generate 484 records.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
****************************** Train MF until converge ******************************
Update bs to 58


******************************MF initialization done ******************************


...

Training // Epoch 18 //  Total cost = -38.3316  Total cost1 = -39.4842  Total cost2 = 1.1526


100%|██████████| 50/50 [00:00<00:00, 161.29it/s]


Training // Epoch 19 //  Total cost = -46.0649  Total cost1 = -47.3458  Total cost2 = 1.2809
****************************** Start simulation ******************************
****************************** Epoch 0 ******************************


100%|██████████| 50/50 [00:00<00:00, 3571.08it/s]


IndexError: index 2678 is out of bounds for axis 0 with size 1000

In [ ]:
# Calculate the average of each corresponding element across lists
print(gini_coefficients)
average_gini_coefficients = np.mean(np.array(gini_coefficients), axis=0)

# Visualize the averaged Gini coefficients
plt.figure(figsize=(10, 6))
x_values = range(len(average_gini_coefficients))
plt.plot(x_values, average_gini_coefficients, marker='o', linestyle='-', color='b')
plt.title('Average Gini Coefficients Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Average Gini Coefficient')
plt.grid(True)
print(average_gini_coefficients)
plt.show()

[]


TypeError: object of type 'numpy.float64' has no len()

<Figure size 1000x600 with 0 Axes>